# Fine Tuning RoBERTa for Toxic Comment Classification (Multi label Text Classification)

## Parameters
### Batch size 32
### Learning Rate - 1e-5
### Loss Function - BCEWithLogitsLoss
### POS_weight
### Remove severe_toxicity
### Dropout 0.4



<a id='section01'></a>
### Importing Python Libraries and preparing the environment

At this step we will be importing the libraries and modules needed to run our script. Libraries are:
* Pandas
* Pytorch
* Pytorch Utils for Dataset and Dataloader
* Transformers
* BERT Model and Tokenizer

Followed by that we will preapre the device for GPU execeution. This configuration is needed if you want to leverage on onboard GPU.

*I have included the code for TPU configuration, but commented it out. If you plan to use the TPU, please comment the GPU execution codes and uncomment the TPU ones to install the packages and define the device.*

In [2]:
#Run this before a fresh training to clear the ouput folder, dont run this if you want to continue from previous training
#!rm -rf /kaggle/input/previous-model/checkpoint_2.pth
#!rm -rf /kaggle/working/*

In [3]:
# Installing the transformers library and additional libraries if looking process

#!pip install upgrade transformers
#!pip install torch

#Code for TPU packages install
# !curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
# !pip install torch==2.1.0

In [4]:
# Importing stock ml libraries
import os
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig, DistilBertTokenizer, DistilBertModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup

#Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()
# device

In [5]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

<a id='section02'></a>
### Importing and Pre-Processing the domain data

We will be working with the data and preparing for fine tuning purposes.
*Assuming that the `train.csv` is already downloaded, unzipped and saved in your `data` folder*

* Import the file in a dataframe and give it the headers as per the documentation.
* Taking the values of all the categories and coverting it into a list.
* The list is appened as a new column and other columns are removed

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')
!ls /datasets/sampled_dataset

test.parquet  train.parquet  validation.parquet


In [7]:
# df1_train = pd.read_parquet("/content/drive/MyDrive/TCC_Data/train-00000-of-00002.parquet")
# df2_train = pd.read_parquet("/content/drive/MyDrive/TCC_Data/train-00001-of-00002.parquet")
# df_validation = pd.read_parquet("/content/drive/MyDrive/TCC_Data/validation-00000-of-00001.parquet")
# df_test = pd.read_parquet("/content/drive/MyDrive/TCC_Data/test-00000-of-00001.parquet")

#Get data from dektop
# df1_train = pd.read_parquet(r"C:\Users\gauta\Documents\LJMU Masters\Toxicity\Experiments\Data\train-00000-of-00002.parquet")
# df2_train = pd.read_parquet(r"C:\Users\gauta\Documents\LJMU Masters\Toxicity\Experiments\Data\train-00001-of-00002.parquet")
# df_validation = pd.read_parquet(r"C:\Users\gauta\Documents\LJMU Masters\Toxicity\Experiments\Data\validation-00000-of-00001.parquet")
# df_test = pd.read_parquet(r"C:\Users\gauta\Documents\LJMU Masters\Toxicity\Experiments\Data\test-00000-of-00001.parquet")

#Get data from dektop
df_train = pd.read_parquet(r"/datasets/sampled_dataset/train.parquet")
#df2_train = pd.read_parquet(r"/datasets/toxic_comments_dataset/train-00001-of-00002.parquet")
df_validation = pd.read_parquet(r"/datasets/sampled_dataset/validation.parquet")
df_test = pd.read_parquet(r"/datasets/sampled_dataset/test.parquet")

# def conv_probs_to_labels(df):
#   df['toxicity'] = df['toxicity'].apply(lambda x: 0 if x <0.5 else 1)
#   df['severe_toxicity'] = df['severe_toxicity'].apply(lambda x: 0 if x <0.2 else 1)
#   df['obscene'] = df['obscene'].apply(lambda x: 0 if x <0.5 else 1)
#   df['threat'] = df['threat'].apply(lambda x: 0 if x <0.5 else 1)
#   df['insult'] = df['insult'].apply(lambda x: 0 if x <0.5 else 1)
#   df['identity_attack'] = df['identity_attack'].apply(lambda x: 0 if x <0.5 else 1)
#   df['sexual_explicit'] = df['sexual_explicit'].apply(lambda x: 0 if x <0.5 else 1)
#   return df

# df1_train = conv_probs_to_labels(df1_train)
# df2_train = conv_probs_to_labels(df2_train)
# df_validation = conv_probs_to_labels(df_validation).reset_index()
# df_test = conv_probs_to_labels(df_test).reset_index()

#df_train = pd.concat([df1_train,df2_train],axis=0).reset_index()
# df_train['list'] = df_train[df_train.columns[1:]].values.tolist()
# df_test['list'] = df_test[df_test.columns[1:]].values.tolist()
# df_validation['list'] = df_validation[df_validation.columns[1:]].values.tolist()
# df_train = df_train[['text', 'list']].copy().reset_index()
# df_test = df_test[['text', 'list']].copy().reset_index()
# df_validation = df_validation[['text', 'list']].copy().reset_index()
# df_train = df_train.head(1000)
# df_test = df_test.head(200)
# df_validation = df_validation.head(200)

In [8]:
print(df_train.toxicity.sum())
#print(df_train.severe_toxicity.sum())
print(df_train.obscene.sum())
print(df_train.threat.sum())
print(df_train.insult.sum())
print(df_train.identity_attack.sum())
print(df_train.sexual_explicit.sum())

96796
6582
3778
71258
8674
3262


In [9]:
df_train.text.str.len().mean()

133.99267619278277

In [10]:
target_cols = ['toxicity','obscene','threat','insult','identity_attack','sexual_explicit']
df_train[target_cols].values
df_train = df_train[['text'] + target_cols].reset_index()
df_test = df_test[['text'] + target_cols].reset_index()
df_validation = df_validation[['text'] + target_cols].reset_index()

<a id='section03'></a>
### Preparing the Dataset and Dataloader

We will start with defining few key variables that will be used later during the training/fine tuning stage.
Followed by creation of CustomDataset class - This defines how the text is pre-processed before sending it to the neural network. We will also define the Dataloader that will feed  the data in batches to the neural network for suitable training and processing.
Dataset and Dataloader are constructs of the PyTorch library for defining and controlling the data pre-processing and its passage to neural network. For further reading into Dataset and Dataloader read the [docs at PyTorch](https://pytorch.org/docs/stable/data.html)

#### *CustomDataset* Dataset Class
- This class is defined to accept the `tokenizer`, `dataframe` and `max_length` as input and generate tokenized output and tags that is used by the BERT model for training.
- We are using the BERT tokenizer to tokenize the data in the `comment_text` column of the dataframe.
- The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs, namely: `ids`, `attention_mask`, `token_type_ids`
---
- *This is the first difference between the distilbert and bert, where the tokenizer generates the token_type_ids in case of Bert*
---
- To read further into the tokenizer, [refer to this document](https://huggingface.co/transformers/model_doc/bert.html#berttokenizer)
- `targest` is the list of categories labled as `0` or `1` in the dataframe.
- The *CustomDataset* class is used to create 2 datasets, for training and for validation.
- *Training Dataset* is used to fine tune the model: **80% of the original data**
- *Validation Dataset* is used to evaluate the performance of the model. The model has not seen this data during training.

#### Dataloader
- Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of dataloaded to the memory and then passed to the neural network needs to be controlled.
- This control is achieved using the parameters such as `batch_size` and `max_len`.
- Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [11]:
# Sections of config

# Defining some key variables that will be used later on in the training
#from transformers import BertTokenizer
MAX_LEN = 300
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32
EPOCHS = 4
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# model_path = '/kaggle/working/checkpoint.pth'
# model_path5 = '/kaggle/working/checkpoint_5.pth'
# model_path4 = '/kaggle/working/checkpoint_4.pth'
# model_path3 = '/kaggle/working/checkpoint_3.pth'
# model_path2 = '/kaggle/working/checkpoint_2.pth'
# model_path1 = '/kaggle/working/checkpoint_1.pth'
#tokenizer = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")

In [12]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = dataframe[target_cols].values
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            #padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [13]:
# Creating the dataset and dataloader for the neural network

#train_size = 0.8
train_dataset=df_train
test_dataset=df_test
validation_dataset=df_validation.reset_index(drop = True)
train_dataset = train_dataset.reset_index(drop=True)


# print("FULL Dataset: {}".format(new_df.shape))
# print("TRAIN Dataset: {}".format(train_dataset.shape))
# print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)
validation_set = CustomDataset(validation_dataset,tokenizer,MAX_LEN)

In [14]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

validation_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
validation_loader = DataLoader(validation_set, **validation_params)

<a id='section04'></a>
### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `BERTClass`.
 - This network will have the `Bert` model.  Follwed by a `Droput` and `Linear Layer`. They are added for the purpose of **Regulariaztion** and **Classification** respectively.
 - In the forward loop, there are 2 output from the `BertModel` layer.
 - The second output `output_1` or called the `pooled output` is passed to the `Drop Out layer` and the subsequent output is given to the `Linear layer`.
 - Keep note the number of dimensions for `Linear Layer` is **6** because that is the total number of categories in which we are looking to classify our model.
 - The data will be fed to the `BertClass` as defined in the dataset.
 - Final layer outputs is what will be used to calcuate the loss and to determine the accuracy of models prediction.
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference.

#### Loss Function and Optimizer
 - The Loss is defined in the next cell as `loss_fn`.
 - As defined above, the loss function used will be a combination of Binary Cross Entropy which is implemented as [BCELogits Loss](https://pytorch.org/docs/stable/nn.html#bcewithlogitsloss) in PyTorch
 - `Optimizer` is defined in the next cell.
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

#### Further Reading
- You can refer to my [Pytorch Tutorials](https://github.com/abhimishra91/pytorch-tutorials) to get an intuition of Loss Function and Optimizer.
- [Pytorch Documentation for Loss Function](https://pytorch.org/docs/stable/nn.html#loss-functions)
- [Pytorch Documentation for Optimizer](https://pytorch.org/docs/stable/optim.html)
- Refer to the links provided on the top of the notebook to read more about `BertModel`.

In [15]:
#Handle class Imbalance with pos_weights
# weight_for_class_i = total_samples / (num_samples_in_class_i * num_classes)
# num_classes = 7
# weights = [len(df_train[i])/(df_train[i].sum() *num_classes) for i in target_cols]
# weights = torch.tensor(weights,device = device)
# print(weights)

# weights = torch.tensor([0.3,15,5,5,0.5,3.5,10],device = device)
# print(weights)
weights  = torch.tensor(5, device = device)

In [16]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.DistilBertModel.from_pretrained('distilbert-base-uncased')
        #self.l1 = AutoModelForSequenceClassification.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")
        self.l2 = torch.nn.Dropout(0.4)
        self.l3 = torch.nn.Linear(768, 6)

    #def forward(self, ids, mask, token_type_ids):
    def forward(self, ids, mask):
        output_1= self.l1(ids, attention_mask = mask, return_dict=True)
        #output_1= self.l1(ids, attention_mask = mask, return_dict=False)
        output_2 = self.l2(output_1.last_hidden_state[:, 0])
        output = self.l3(output_2)
        #logits = self.fc(output)
        return output
    
def loss_fn(outputs, targets, weights):
    loss_func = torch.nn.BCEWithLogitsLoss(pos_weight = weights) #pos_weight scalar
    return loss_func(outputs, targets)
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClass()
optimizer = torch.optim.AdamW(params =  model.parameters(), lr=LEARNING_RATE)
total_steps = len(training_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
model.to(device)

start_epoch = 0
previous_checkpoint = '/kaggle/input/previous-model-checkpoint/checkpoint.pth'
if os.path.exists(previous_checkpoint):
    checkpoint = torch.load(previous_checkpoint)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    start_epoch = epoch+1
    print(f"Loaded checkpoint from epoch {epoch}")
else:
    print("No checkpoint found, starting from scratch.")
    #model = BERTClassifier(bert_model_name, num_classes)
    #model = BERTClass()

# for name, param in model.named_parameters():
#     if name.startswith('bert.encoder.layer.1'):
#         param.requires_grad = False
#     if name.startswith('bert.encoder.layer.2'):
#         param.requires_grad = False
#     if name.startswith('bert.encoder.layer.3'):
#         param.requires_grad = False
#     if name.startswith('bert.encoder.layer.4'):
#         param.requires_grad = False
#     if name.startswith('bert.encoder.layer.5'):
#         param.requires_grad = False
#     if name.startswith('bert.encoder.layer.6'):
#         param.requires_grad = False
#     if name.startswith('bert.encoder.layer.7'):
#         param.requires_grad = False
#     if name.startswith('bert.encoder.layer.8'):
#         param.requires_grad = False
#     if name.startswith('bert.encoder.layer.9'):
#         param.requires_grad = False

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


No checkpoint found, starting from scratch.


<a id='section05'></a>
### Fine Tuning the Model

After all the effort of loading and preparing the data and datasets, creating the model and defining its loss and optimizer. This is probably the easier steps in the process.

Here we define a training function that trains the model on the training dataset created above, specified number of times (EPOCH), An epoch defines how many times the complete data will be passed through the network.

Following events happen in this function to fine tune the neural network:
- The dataloader passes data to the model based on the batch size.
- Subsequent output from the model and the actual category are compared to calculate the loss.
- Loss value is used to optimize the weights of the neurons in the network.
- After every 5000 steps the loss value is printed in the console.

As you can see just in 1 epoch by the final step the model was working with a miniscule loss of 0.022 i.e. the network output is extremely close to the actual output.

In [17]:
def train(epoch,batch_size,weights):
    model.train()

    total_loss, total_accuracy = 0, 0

    total = len(training_loader)
    
    fin_targets=[]
    fin_outputs=[]

    for _,data in enumerate(training_loader, 0):

        step = _+1
        percent = "{0:.10f}".format(100 * (step / float(total)))
        lossp = "{0:.10f}".format(total_loss/(total*batch_size))
        filledLength = int(100 * step // total)
        bar = '█' * filledLength + '>'  *(filledLength < 100) + '.' * (99 - filledLength)
        print(f'\rBatch {step}/{total} |{bar}| {percent}% complete, loss={lossp}, accuracy={total_accuracy}', end='')


        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        #outputs = model(ids, mask, token_type_ids)
        outputs = model(ids, mask)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets,weights)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        fin_targets.extend(targets.cpu().detach().numpy().tolist())
        fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
        

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return fin_outputs, fin_targets, loss

In [18]:
# for epoch in range(EPOCHS):
#     print('Epoch',epoch+1,'of',EPOCHS)
#     train(epoch,8)

<a id='section06'></a>
### Validating the Model

During the validation stage we pass the unseen data(Testing Dataset) to the model. This step determines how good the model performs on the unseen data.

This unseen data is the 20% of `train.csv` which was seperated during the Dataset creation stage.
During the validation stage the weights of the model are not updated. Only the final output is compared to the actual value. This comparison is then used to calcuate the accuracy of the model.

As defined above to get a measure of our models performance we are using the following metrics.
- Accuracy Score
- F1 Micro
- F1 Macro

We are getting amazing results for all these 3 categories just by training the model for 1 Epoch.

In [19]:
def validation(model,data_loader,device,weights):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(data_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            #outputs = model(ids, mask, token_type_ids)
            outputs = model(ids, mask)
            loss = loss_fn(outputs, targets,weights)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets,loss

In [ ]:
from collections import defaultdict
history = defaultdict(list)

for epoch in range(start_epoch, EPOCHS):
    print('Epoch',epoch+1,'of',EPOCHS)
    train_op, train_target,train_loss = train(epoch,TRAIN_BATCH_SIZE,weights)
    val_op, val_target,val_loss = validation(model,validation_loader,device,weights)
    train_op = np.array(train_op) >= 0.5
    
    val_op = np.array(val_op) >= 0.5
    train_accuracy = round(metrics.accuracy_score(train_target, train_op),3)
    train_precision_score_micro = round(metrics.precision_score(train_target, train_op, average='micro'),3)
    train_precision_score_macro = round(metrics.precision_score(train_target, train_op, average='macro'),3)
    train_recall_score_micro = round(metrics.recall_score(train_target, train_op, average='micro'),3)
    train_recall_score_macro = round(metrics.recall_score(train_target, train_op, average='macro'),3)
    train_f1_score_micro = round(metrics.f1_score(train_target, train_op, average='micro'),3)
    train_f1_score_macro = round(metrics.f1_score(train_target, train_op, average='macro'),3)
    train_auc_roc_micro = round(metrics.roc_auc_score(train_target, train_op, average='micro'),3)
    train_auc_roc_macro = round(metrics.roc_auc_score(train_target, train_op, average='macro'),3)
    train_clf = metrics.classification_report(train_target, train_op)
    
    val_accuracy = round(metrics.accuracy_score(val_target, val_op),3)
    val_precision_score_micro = round(metrics.precision_score(val_target, val_op, average='micro'),3)
    val_precision_score_macro = round(metrics.precision_score(val_target, val_op, average='macro'),3)
    val_recall_score_micro = round(metrics.recall_score(val_target, val_op, average='micro'),3)
    val_recall_score_macro = round(metrics.recall_score(val_target, val_op, average='macro'),3)
    val_f1_score_micro = round(metrics.f1_score(val_target, val_op, average='micro'),3)
    val_f1_score_macro = round(metrics.f1_score(val_target, val_op, average='macro'),3)
    val_auc_roc_micro = round(metrics.roc_auc_score(val_target, val_op, average='micro'),3)
    val_auc_roc_macro = round(metrics.roc_auc_score(val_target, val_op, average='macro'),3)
    val_clf = metrics.classification_report(val_target,val_op)
    
#     if os.path.exists(model_path):
#         os.remove(model_path)  #Delete the previous file if any
    path = 'Output/checkpoint_' + str(epoch+1)+'.pth'
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': train_loss,
            }, path)
    
    
    history['train_acc'].append(train_accuracy)
    history['train_loss'].append(round(train_loss.item(),4))
    history['train_precision_score_micro'].append(train_precision_score_micro)
    history['train_precision_score_macro'].append(train_precision_score_macro)    
    history['train_recall_score_micro'].append(train_recall_score_micro)
    history['train_recall_score_macro'].append(train_recall_score_macro)
    history['train_f1_score_micro'].append(train_f1_score_micro)
    history['train_f1_score_macro'].append(train_f1_score_macro)
    history['train_rocauc_score_micro'].append(train_auc_roc_micro)
    history['train_rocauc_score_macro'].append(train_auc_roc_macro)
    
    history['val_acc'].append(val_accuracy)
    history['val_loss'].append(round(val_loss.item(),4))
    history['val_precision_score_micro'].append(val_precision_score_micro)
    history['val_precision_score_macro'].append(val_precision_score_macro)
    history['val_recall_score_micro'].append(val_recall_score_micro)
    history['val_recall_score_macro'].append(val_recall_score_macro)
    history['val_f1_score_micro'].append(val_f1_score_micro)
    history['val_f1_score_macro'].append(val_f1_score_macro)  
    history['val_rocauc_score_micro'].append(val_auc_roc_micro)
    history['val_rocauc_score_macro'].append(val_auc_roc_macro)
    
    print(f"Train Accuracy Score = {train_accuracy}")
    print(f"Train Precision Score (Micro) = {train_precision_score_micro}")
    print(f"Train Precision Score (Micro) = {train_precision_score_macro}")
    print(f"Train Recall Score (Micro) = {train_recall_score_micro}")
    print(f"Train Recall Score (Macro) = {train_recall_score_macro}")
    print(f"Train F1 Score (Micro) = {train_f1_score_micro}")
    print(f"Train F1 Score (Macro) = {train_f1_score_macro}")
    print(f"Train AUC-ROC Score (Micro) = {train_auc_roc_micro}")
    print(f"Train AUC-ROC Score (Macro) = {train_auc_roc_macro}")
    print("Training Classification Report")
    print("==============================")
    print(train_clf)
    
    print(f"Validation Accuracy Score = {val_accuracy}")
    print(f"Validation Precision Score (Micro) = {val_precision_score_micro}")
    print(f"Validation Precision Score (Micro) = {val_precision_score_macro}")
    print(f"Validation Recall Score (Micro) = {val_recall_score_micro}")
    print(f"Validation Recall Score (Macro) = {val_recall_score_macro}")
    print(f"Validation F1 Score (Micro) = {val_f1_score_micro}")
    print(f"Validation F1 Score (Macro) = {val_f1_score_macro}")
    print(f"Validation AUC-ROC Score (Micro) = {val_auc_roc_micro}")
    print(f"Validation AUC-ROC Score (Macro) = {val_auc_roc_macro}")
    print("Validation Classification Report")
    print("================================")
    print(val_clf)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch 1 of 4
Batch 1/6452 |>...................................................................................................| 0.0154990701% complete, loss=0.0000000000, accuracy=0Epoch: 0, Loss:  1.0738698244094849
Batch 5001/6452 |█████████████████████████████████████████████████████████████████████████████>......................| 77.5108493490% complete, loss=0.0000000000, accuracy=0Epoch: 0, Loss:  0.3561294972896576
Batch 6452/6452 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0000000000% complete, loss=0.0000000000, accuracy=0

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1

Train Accuracy Score = 0.721
Train Precision Score (Micro) = 0.702
Train Precision Score (Micro) = 0.578
Train Recall Score (Micro) = 0.944
Train Recall Score (Macro) = 0.829
Train F1 Score (Micro) = 0.805
Train F1 Score (Macro) = 0.68
Train AUC-ROC Score (Micro) = 0.936
Train AUC-ROC Score (Macro) = 0.866
Training Classification Report
              precision    recall  f1-score   support

           0       0.76      0.97      0.85     96796
           1       0.57      0.81      0.67      6582
           2       0.43      0.69      0.53      3778
           3       0.69      0.96      0.80     71258
           4       0.52      0.81      0.63      8674
           5       0.50      0.74      0.59      3262

   micro avg       0.70      0.94      0.81    190350
   macro avg       0.58      0.83      0.68    190350
weighted avg       0.71      0.94      0.81    190350
 samples avg       0.42      0.45      0.43    190350

Validation Accuracy Score = 0.782
Validation Precision Score (Mi

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


...

Batch 5001/6452 |█████████████████████████████████████████████████████████████████████████████>......................| 77.5108493490% complete, loss=0.0000000000, accuracy=0Epoch: 2, Loss:  0.1778019666671753
Batch 6452/6452 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0000000000% complete, loss=0.0000000000, accuracy=0

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1

Train Accuracy Score = 0.794
Train Precision Score (Micro) = 0.77
Train Precision Score (Micro) = 0.656
Train Recall Score (Micro) = 0.974
Train Recall Score (Macro) = 0.932
Train F1 Score (Micro) = 0.86
Train F1 Score (Macro) = 0.767
Train AUC-ROC Score (Micro) = 0.961
Train AUC-ROC Score (Macro) = 0.932
Training Classification Report
              precision    recall  f1-score   support

           0       0.83      0.99      0.90     96796
           1       0.62      0.92      0.74      6582
           2       0.55      0.89      0.68      3778
           3       0.76      0.98      0.86     71258
           4       0.58      0.92      0.71      8674
           5       0.59      0.90      0.71      3262

   micro avg       0.77      0.97      0.86    190350
   macro avg       0.66      0.93      0.77    190350
weighted avg       0.78      0.97      0.86    190350
 samples avg       0.43      0.47      0.44    190350

Validation Accuracy Score = 0.777
Validation Precision Score (Mic

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Batch 5001/6452 |█████████████████████████████████████████████████████████████████████████████>......................| 77.5108493490% complete, loss=0.0000000000, accuracy=0Epoch: 3, Loss:  0.18633423745632172
Batch 6452/6452 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0000000000% complete, loss=0.0000000000, accuracy=0

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history['train_acc'], label='train accuracy')
plt.plot(history['val_acc'], label='validation accuracy')
plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1]);
plt.show()

plt.plot(history['train_loss'], label='training loss')
plt.plot(history['val_loss'], label='validation loss')
plt.title('Training history')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1]);
plt.show()

In [ ]:
history

In [ ]:
test_op,test_target, test_loss = validation(
  model,
  testing_loader,
  device,weights
)

test_op = np.array(test_op) >= 0.5
test_accuracy = metrics.accuracy_score(test_target, test_op)
test_precision_score_micro = metrics.precision_score(test_target, test_op, average='micro')
test_precision_score_macro = metrics.precision_score(test_target, test_op, average='macro')
test_recall_score_micro = metrics.recall_score(test_target, test_op, average='micro')
test_recall_score_macro = metrics.recall_score(test_target, test_op, average='macro')
test_f1_score_micro = metrics.f1_score(test_target, test_op, average='micro')
test_f1_score_macro = metrics.f1_score(test_target, test_op, average='macro')
test_clf = metrics.classification_report(test_target, test_op)
test_roc_auc_micro = metrics.roc_auc_score(test_target,test_op, average='micro')
test_roc_auc_macro = metrics.roc_auc_score(test_target,test_op, average='macro')

print('accuracy',test_accuracy)
print('precision micro',test_precision_score_micro)
print('precision macro',test_precision_score_macro)
print('recall micro',test_recall_score_micro)
print('recall macro',test_recall_score_macro)
print('f1 micro',test_f1_score_micro)
print('f1 macro',test_f1_score_macro)
print('roc_auc micro',test_roc_auc_micro)
print('roc_auc macro',test_roc_auc_macro)
print('Classification Report')
print('=====================')
print(test_clf)

In [ ]:
y_pred, y_true,loss = validation(
  model,
  testing_loader,device,weights
)
y_pred = np.array(y_pred) >= 0.5
y_pred

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_true, y_pred))

In [ ]:
#from sklearn.metrics import multilabel_confusion_matrix
confusion_matrix(np.array(y_true).argmax(axis=1),np.array(y_pred).argmax(axis=1))

In [ ]:
import seaborn as sns
class_name = ['toxic','obscene','threat','insult','identity_hate','sexual_explicit']
def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True sentiment')
  plt.xlabel('Predicted sentiment');
cm = confusion_matrix(np.array(y_true).argmax(axis=1),np.array(y_pred).argmax(axis=1))
df_cm = pd.DataFrame(cm, index=class_name, columns=class_name)
show_confusion_matrix(df_cm)

In [ ]:
#Prediction

In [ ]:
class_name = ['toxic','obscene','threat','insult','identity_hate','sexual_explicit']

review_text = "You criminal.. I will k*ll you"

encoded_review = tokenizer.encode_plus(
  review_text,
  max_length=100,
  add_special_tokens=True,
  return_token_type_ids=True,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',
)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

input_ids = encoded_review['input_ids'].to(device)
attention_mask = encoded_review['attention_mask'].to(device)
token_type_ids = encoded_review['token_type_ids'].to(device)
output = model(input_ids, attention_mask)
output = sigmoid(output.cpu().detach())>0.5
output = list(output)

print(f'Review text: {review_text}')
print(f'Sentiment  : {output}')

op = {
    'toxic':output[0][0],
    'obscene':output[0][1],
    'threat':output[0][2],
    'insult':output[0][3],
    'identity_hate':output[0][4],
    'sexual_explicit':output[0][5]
}
op

<a id='section07'></a>
### Saving the Trained Model Artifacts for inference

This is the final step in the process of fine tuning the model.

The model and its vocabulary are saved locally. These files are then used in the future to make inference on new inputs of news headlines.

Please remember that a trained neural network is only useful when used in actual inference after its training.

In the lifecycle of an ML projects this is only half the job done. We will leave the inference of these models for some other day.